## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Fashion House Synthetic Data with Generative AI

### License

In [ ]:
##################################################################################
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview


Use LLMs for code generation.  You can start with a table schema with basic field descriptions

1.   Create your table DDLs
2.   Create LLM prompts for each table and ask it to populate the table with data
3. Provide the prompts with starting primary keys
4. Provide the prompts with foreign keys
5. The LLM can understand that it decides the sizes and prices according to the item where applicable
6. The LLM can read the description of each field and use that to generate valid values

### Initialize Python

In [ ]:
import pandas as pd
import json
import bigframes.pandas as bf
#from bigframes.llm import BigFramesLLM
from bigframes.ml.llm import PaLM2TextGenerator

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
PROJECT_ID = "${project_id}"
REGION = "us"
DATASET_ID = "${dataset_id}"
CONNECTION_NAME = "vertex-ai"

connection = f"{PROJECT_ID}.{REGION}.{CONNECTION_NAME}"

In [ ]:
session = bf.get_global_session()

llm_model = PaLM2TextGenerator(session=session, connection_name=connection)

### Supporting Functions

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

In [ ]:
def LLM(prompt, isOutputJson, max_output_tokens=1024, temperature=0, top_p=0, top_k=1):
  print()
  print("Prompt: ", prompt)
  print()
  df_prompt = pd.DataFrame(
          {
              "prompt": [prompt],
          })
  bf_df_prompt = bf.read_pandas(df_prompt)
  prediction = llm_model.predict(bf_df_prompt,
                                 max_output_tokens=max_output_tokens,
                                 temperature=temperature, # 0 to 1 (1 random)
                                 top_p=top_p, # 0 to 1 (1 random)
                                 top_k=top_k, # (1 to 40 random)
                                 ).to_pandas()
  try:
    # Remove common LLM output mistakes
    result = prediction['ml_generate_text_llm_result'][0]

    result = result.replace("```json\n","")
    result = result.replace("```JSON\n","")
    result = result.replace("```json","")
    result = result.replace("```JSON","")
    result = result.replace("```sql\n","")
    result = result.replace("```SQL\n","")
    result = result.replace("```sql","")
    result = result.replace("```SQL","")
    result = result.replace("```","")

    if isOutputJson:
      result = result.replace("\n"," ")
      json_string = PrettyPrintJson(result)
      json_string = json_string.replace("'","\\'")
      json_string = json_string.strip()
      return json_string
    else:
      if "INSERT INTO" in result:
        # do nothing (do not escape the single ticks, the LLM should do this
        #             automatically for any text fields)
        print("Do nothing")
      else:
        result = result.replace("'","\\'")
      result = result.strip()
      return result

  except:
    print("Error (raw): ", prediction['ml_generate_text_llm_result'][0])
    print("Error (result): ", result)


In [ ]:
def GetTableSchema(dataset_name, table_name):
  import io

  dataset_ref = client.dataset(dataset_name, project=PROJECT_ID)
  table_ref = dataset_ref.table(table_name)
  table = client.get_table(table_ref)

  f = io.StringIO("")
  client.schema_to_json(table.schema, f)
  return f.getvalue()

In [ ]:
def GetForeignKeys(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(CAST({field_name} AS STRING), "," ORDER BY {field_name}) AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetDistinctValues(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(DISTINCT {field_name}, "," ) AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetStartingValue(dataset_name, table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) + 1 AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetMaximumValue(dataset_name, table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def RunQuery(sql):
  import time

  #return True # return early for now

  job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
  query_job = client.query(sql, job_config=job_config)

  # Check on the progress by getting the job's updated state.
  query_job = client.get_job(
      query_job.job_id, location=query_job.location
  )
  print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  while query_job.state != "DONE":
    time.sleep(2)
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
        )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  if query_job.error_result == None:
    return True
  else:
    return False

### Create Tables

- Here we are starting with our schema DDL and using our description to let the LLM know valid values.

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.customer`
(
    customer_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    customer_name STRING NOT NULL OPTIONS(description="Name of the customer."),
    customer_yob INT NOT NULL OPTIONS(description="Customer year of birth"),
    customer_email STRING NOT NULL OPTIONS(description="Customer's email address"),
    customer_inception_date DATE NOT NULL OPTIONS(description="Date of first customer interaction")

)
CLUSTER BY customer_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.customer_review`
(
    customer_review_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    customer_id INTEGER NOT NULL OPTIONS(description="Foreign key: Customer table"),
    review_datetime TIMESTAMP NOT NULL OPTIONS(description="Date and time of the review."),
    review_text STRING NOT NULL OPTIONS(description="The customer's review of the product."),
    review_audio STRING OPTIONS(description="The GCS location of an attached Audio file."),
    review_sentiment STRING OPTIONS(description="The sentiment of the review text."),
    social_media_source STRING NOT NULL OPTIONS(description="The social media site the review was posted on."),
    social_media_handle STRING NOT NULL OPTIONS(description="The customer's social media handle"),
    gen_ai_recommended_action STRING OPTIONS(description="Valid values for gen_ai_recommended_action are: 'Send Survey', 'Send Coupon'"),
    gen_ai_reponse STRING OPTIONS(description="The Generated response from the LLM."),
    human_response STRING OPTIONS(description="The human manually entered response."),
    response_sent_action STRING OPTIONS(description="Valid values for response_sent_action are: 'Sent LLM Response', 'Human called'"),
    response_sent_date TIMESTAMP OPTIONS(description="Date and time the response was sent.")

)
CLUSTER BY customer_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""ALTER TABLE `{PROJECT_ID}.{DATASET_ID}.customer_review`
  ADD COLUMN IF NOT EXISTS llm_detected_theme JSON OPTIONS(description="The LLM detected themes in the customer review.");"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.product`
(
    product_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    product_name STRING NOT NULL OPTIONS(description="The name of the product apparel or accessory"),
    product_price FLOAT64 NOT NULL OPTIONS(description="The price of the product"),
    product_description STRING NOT NULL OPTIONS(description="The detailed description of the product"),
    item_size STRING NOT NULL OPTIONS(description="Valid Values: Small, Medium, Large")

)
CLUSTER BY product_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.order`
(
    order_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    customer_id INTEGER NOT NULL OPTIONS(description="Foreign key: Customer table."),
    order_datetime TIMESTAMP NOT NULL OPTIONS(description="The datetime the order was started."),
    order_completion_datetime TIMESTAMP NOT NULL OPTIONS(description="The datetime the order was completed.")

)
CLUSTER BY order_id;
"""

client.query(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.order_item`
(
    order_item_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    order_id INTEGER NOT NULL OPTIONS(description="Foreign key: Order table"),
    product_id INTEGER NOT NULL OPTIONS(description="Foreign key: Product table"),
    quantity INTEGER NOT NULL OPTIONS(description="Number of items ordered")

)
CLUSTER BY order_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.historical_sales`
(
    order_item_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    order_id INTEGER NOT NULL OPTIONS(description="Foreign key: Order table"),
    product_id INTEGER NOT NULL OPTIONS(description="Foreign key: Product table"),
    quantity INTEGER NOT NULL OPTIONS(description="Number of items ordered"),
    price FLOAT64 NOT NULL OPTIONS(description="Price of the product"),
    order_datetime TIMESTAMP NOT NULL OPTIONS(description="The datetime the order was completed."),
    total_sales_amount FLOAT64 NOT NULL OPTIONS(description="Total sales amount for this order, multipling quantity by price.")
)
CLUSTER BY order_id;
"""

RunQuery(sql)


### Customer Table

In [ ]:
sql = f"""ALTER TABLE `{PROJECT_ID}.{DATASET_ID}.customer`
  ADD COLUMN IF NOT EXISTS country_code STRING OPTIONS(description="The home country of the customer.");"""

RunQuery(sql)

In [ ]:
customer_count = 10
country = "Sweden"
country_code = "SE"

table_name = "customer"
primary_key = "customer_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 1
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {customer_count} insert statements for this table.
  - The customer_inception_date is a date and should be within the past 2 years.
  - The customer_name should be names used in the country {country} and be a first and last name.
  - The starting value of the field {primary_key} is {starting_value}.
  - Only generate a single statement, not multiple INSERTs.
  - Set the country_code to {country_code}

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """


  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

### Customer Review Table

### Create customer reviews

In [ ]:
rows_of_data_to_generate = 3

table_name = "customer"
field_name = "customer_id"
customer_ids = GetForeignKeys(DATASET_ID, table_name, field_name)

table_name = "customer_review"
primary_key = "customer_review_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
import random
loop_count = 10
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  if random.random() < .25:
    prompt=f"""
    You are a database engineer and need to generate data for a table for the below schema.
    You need to generate reviews for customers who have purchased your clothing or accessories.
    Write a negative in first person based upon the following: "Bad Quality","Long Delivery Time","Dirty","Overpriced","Not great options","Clothes not according to weather","Lack of options","Inconsistent Quality","Lack of knowledgable staff","No Matching accessories","Not true to size","Limited Options","Delayed Delivery","Return not straight forward"
    - The schema is for a Google Cloud BigQuery Table.
    - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
    - Read the description of each field for valid values.
    - Do not preface the response with any special characters or 'sql'.
    - Generate {rows_of_data_to_generate} insert statements for this table.
    - Valid values for customer_id are: {customer_ids}
    - The review_datetime is a date and should be within the past year.
    - The response for each question should be 20 to 100 words.
    - The starting value of the field {primary_key} is {starting_value}.
    - Only generate a single statement, not multiple INSERTs.
    - Escape single quotes with a backslash.  Example: Kim's Answer: Kim\'s
    - Only generate data for these fields: customer_review_id, customer_id, review_datetime, review_text, social_media_source, social_media_handle

    Examples:
    Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
    Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

    Schema: {schema}
    """
  else:
    prompt=f"""
    You are a database engineer and need to generate data for a table for the below schema.
    You need to generate reviews for customers who have purchased your clothing or accessories.
    Write a positive or neutral review in first person based upon the following: "Good Quality","Short Delivery Time","Clean","Good value","Great Options","Clothes according to weather","Great matching accessory options","Consistent Quality","True to size","Timely delivery","Easy return options","Good Selection"
    - The schema is for a Google Cloud BigQuery Table.
    - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
    - Read the description of each field for valid values.
    - Do not preface the response with any special characters or 'sql'.
    - Generate {rows_of_data_to_generate} insert statements for this table.
    - Valid values for customer_id are: {customer_ids}
    - The review_datetime is a date and should be within the past year.
    - The response for each question should be 20 to 100 words.
    - The starting value of the field {primary_key} is {starting_value}.
    - Only generate a single statement, not multiple INSERTs.
    - Escape single quotes with a backslash.  Example: Kim's Answer: Kim\'s
    - Only generate data for these fields: customer_review_id, customer_id, review_datetime, review_text, social_media_source, social_media_handle

    Examples:
    Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
    Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

    Schema: {schema}
    """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

### Score the Sentiment

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `{PROJECT_ID}.{DATASET_ID}.customer_review`
         WHERE review_sentiment IS NULL
        ORDER BY customer_review_id
"""

# Fields to update
# review_sentiment,
# gen_ai_recommended_action,
# gen_ai_reponse,
# human_response,
# response_sent_action,
# response_sent_date

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt=f"""
      For the given review classify the sentiment as Positive, Neutral or Negative.
      Review: {review_text}
      """
      review_sentiment = LLM(prompt, False, max_output_tokens=10, temperature=0, top_p=0, top_k=1)

      sql = f"""UPDATE `{PROJECT_ID}.{DATASET_ID}.customer_review`
                  SET review_sentiment = '{review_sentiment}'
                WHERE customer_review_id = {customer_review_id}
      """

      print (sql)

      llm_valid_execution = RunQuery(sql)
      llm_valid_execution = True
    except Exception as error:
      print("An error occurred:", error)

### Gen AI Response

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `{PROJECT_ID}.{DATASET_ID}.customer_review`
         WHERE gen_ai_reponse IS NULL
        ORDER BY customer_review_id"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt=f"""
      Generate responses to the below customer review who purchased clothing or accessories and return the results the below json format.
      The review can be positive, negative, or neutral.
      Provide a variety of responses, including thanking customers for positive reviews, addressing concerns in negative reviews, and engaging with neutral reviews.
      Please generate at least 5 different responses.

      JSON format: [ "value" ]
      Sample JSON Response: [ "response 1", "response 2", "response 3", "response 4", "response 5" ]

      Review: {review_text}"""

      json_result = LLM(prompt, True, max_output_tokens=1024, temperature=0, top_p=0, top_k=1)
      print(f"json_result: {json_result}")

      if json_result == None:
        llm_valid_execution = False
      else:
        sql = f"""UPDATE `{PROJECT_ID}.{DATASET_ID}.customer_review`
                    SET gen_ai_reponse = '{json_result}'
                  WHERE customer_review_id = {customer_review_id}
        """

        print(f"sql: {sql}")

        llm_valid_execution = RunQuery(sql)
    except Exception as error:
      print("An error occurred:", error)

### Gen AI Recommended Action

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `{PROJECT_ID}.{DATASET_ID}.customer_review`
         WHERE gen_ai_recommended_action IS NULL
        ORDER BY customer_review_id"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt="""
      Select one of the following actions based upon the below customer review who purchased clothing or accessories.
      - First randomly sort the actions.
      - Select the best action based upon the sentiment of the review.
      - It is okay to use the action "Send the customer a coupon" for both positive and negative reviews.
      - Return the results the below json format.
      - Do not include any special characters or "```json" in the json output

      Actions
      - "Thank the Customer"
      - "Apologize to the Customer"
      - "Send the customer a coupon"
      - "Call the customer"
      - "Promote Additional Products"
      - "Promise to Investigate"
      - "Encourage More Reviews"
      - "Invite Further Engagement"
      - "Reshare the review on other social media"

      JSON format: { "action" : "value", "explaination" : "llm explaination" }
      Sample JSON Response: { "action" : "Call the customer", "explaination" : "The customer left their phone number in the review." }
      Sample JSON Response: { "action" : "Encourage More Reviews", "explaination" : "Thanks for the review, please keep posting." }

      Review:"""
      prompt = prompt + review_text

      json_result = LLM(prompt, True, max_output_tokens=1024, temperature=0, top_p=0, top_k=1)
      print(f"json_result: {json_result}")

      if json_result == None:
        llm_valid_execution = False
      else:
        sql = f"""UPDATE `{PROJECT_ID}.{DATASET_ID}.customer_review`
                    SET gen_ai_recommended_action = '{json_result}'
                  WHERE customer_review_id = {customer_review_id}
        """

        print(f"sql: {sql}")

        llm_valid_execution = RunQuery(sql)
    except Exception as error:
      print("An error occurred:", error)

### Detect Customer Themes

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `{PROJECT_ID}.{DATASET_ID}.customer_review`
         WHERE llm_detected_theme IS NULL
        ORDER BY customer_review_id"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt="""
      Classify the below customer review as one or more of the below themes.
      - Return the results the below json format.
      - Include an explaination for selecting each theme.
      - Do not include double quotes in the explaination.
      - Do not include any special characters, double quotes or "```json" in the json output.

      Themes
      - "Bad Service"
      - "Long Delivery Time"
      - "Slow Service"
      - "Dirty"
      - "Overpriced"
      - "Overcrowded"
      - "Noisy Location"
      - "Lack of options available"
      - "Inconsistent Quality"
      - "Not true to size"
      - "No Variety of accessories"
      - "Too few options"
      - "Options not according to weather"
      - "Good Service"
      - "Short Delivery Time"
      - "Knowledgeable Staff"
      - "Clean"
      - "Good value"
      - "Value for moeny"
      - "Quite Location"
      - "Variety of Alternatives"
      - "Consistent Quality"
      - "Lots of matching accessories"
      - "Lots of weather appropriate options"
      - "Easy returns"
      - "Good Online Selection"

      JSON format: [{ "theme" : "value", "explaination" : "llm explaination" }]
      Sample JSON Response: [{ "theme" : "Fast Service", "explaination" : "The customer got their order fast." }]
      Sample JSON Response: [{ "theme" : "Overpriced", "explaination" : "The customer said it was too expensive." }]

      Review:"""
      prompt = prompt + review_text

      json_result = LLM(prompt, True, max_output_tokens=1024, temperature=0, top_p=0, top_k=1)
      print(f"json_result: {json_result}")

      if json_result == None:
        llm_valid_execution = False
      else:
        sql = f"""UPDATE `{PROJECT_ID}.{DATASET_ID}.customer_review`
                    SET llm_detected_theme = JSON'{json_result}'
                  WHERE customer_review_id = {customer_review_id}
        """

        print(f"sql: {sql}")

        llm_valid_execution = RunQuery(sql)
    except Exception as error:
      print("An error occurred:", error)

### Product Table

In [ ]:
table_name = "product"
primary_key = "product_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 1
loop_index = 1
product_count = 20

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")

  # Get Product Names
  product_count = 20

  table_name = "product"
  field_name = "product_name"
  existing_values = GetDistinctValues(DATASET_ID, table_name, field_name)

  prompt = f"""Generate {product_count} different clothing and accessory names for spring weather and return in the below json format.
  - The name can be an existing clothing or something new.
  - The name can be an existing accessory or something new.
  - The name should be related to fashion in Nordics with Spring weather as theme.
  - Do not use any of these names: [{existing_values}]
  - Do not number the results.

  JSON format: [ "value" ]
  Sample JSON Response: [ "value1", "value2" ]
  """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      product_name = LLM(prompt, True, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      llm_valid_execution = True
    except Exception as error:
      print("An error occurred:", error)


  # Insert data
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Valid values for item_name are: {product_name}
  - The starting value of the field {primary_key} is {starting_value}.
  - Only generate a single statement, not multiple INSERTs.

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

### Order Table

In [ ]:
order_count = 10

table_name = "customer"
field_name = "customer_id"
max_customer_id = GetMaximumValue(DATASET_ID, table_name, field_name)

table_name = "order"
primary_key = "order_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 50
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {order_count} insert statements for this table.
  - The order_datetime is a date and should be within the past 1 year.
  - The order_completion_datetime should be within 60 to 900 seconds of the order_datetime.
  - Valid values for customer_id between 1 and {max_customer_id}.
  - The starting value of the field {primary_key} is {starting_value}.
  - Only generate a single statement, not multiple INSERTs.
  - Timestamps should use this format: 2020-06-02 23:57:12.120174 UTC.

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      sql = sql.replace("\\'","'")
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)


### Order Item Table

In [ ]:
order_item_count = 3

In [ ]:
table_name = "product"
field_name = "product_id"
max_product_id = GetMaximumValue(DATASET_ID, table_name, field_name)

table_name = "order"
field_name = "order_id"
max_order_id = GetMaximumValue(DATASET_ID, table_name, field_name)

table_name = "order_item"
primary_key = "order_item_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 100
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  max_order_item_id = GetMaximumValue(DATASET_ID, table_name, "order_id")

  if max_order_item_id > max_order_id:
    print("Breaking out of loop since we have items for each order.")
    break

  order_id_to_generate_data = max_order_item_id + 1
  print(f"order_id_to_generate_data: {order_id_to_generate_data}")

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {order_item_count} insert statements for this table.
  - You can have 1 to 10 items for a single order_id.
  - The order_id should use the value of: {order_id_to_generate_data}.
  - Valid values for quantity as between 1 and 4.
  - The starting value of the field {primary_key} is {starting_value}.
  - Only generate a single statement, not multiple INSERTs.

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """


  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

### Historical Sales Table

In [ ]:
order_item_count = 3

In [ ]:
table_name = "trending_product"
field_name = "product_id"
##Add restriction from trending product for product id in prompt

table_name = "order"
field_name = "order_id"
max_order_id = GetMaximumValue(DATASET_ID, table_name, field_name)

table_name = "historical_sales"
primary_key = "order_item_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 100
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  max_order_item_id = GetMaximumValue(DATASET_ID, table_name, "order_id")

  if max_order_item_id > max_order_id:
    print("Breaking out of loop since we have items for each order.")
    break

  order_id_to_generate_data = max_order_item_id + 1
  print(f"order_id_to_generate_data: {order_id_to_generate_data}")

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - This table is to have historical sales data for last year to be used for BigQuery ARIMA PLUS forecasting model.
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {order_item_count} insert statements for this table.
  - You can have 1 to 10 items for a single order_id.
  - The order_id should use the value of: {order_id_to_generate_data}.
  - Valid values for product_id 1 and 2. Do not include more values.
  - The order_datetime is a date and should be between 2023-04-01 and 2024-04-15, generate order for each day.
  - Valid values for quantity as between 1 and 100.
  - The starting value of the field {primary_key} is {starting_value}.
  - Only generate a single statement, not multiple INSERTs.

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """


  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

### Learnings

Issues:
- LLMs take time for each call
- This is a demo to create small dataset
- Output contains: ```
- Output contains: ```sql
- dates are not always valid dates "2017-09-31" to type DATE
- for UUIDs, use an INT and then swap to UUID later (add a column and then do an update)
- LLM returns single quotes in strings.  Had to prompt or string.replace.
- Probally need to use Min/Max of primary key for ints.
- Sometimes the LLM generates multiple insert..intos
- Inserts are sometimes many INSERTS and not many values

Learnings
- The LLM can generate Small, Medium and Large (for applicable products) with pricing that is correct.
- The LLM can understand the schema
- The LLM can understand the description (valid values)